In [1]:
import tensorflow as tf
import numpy as np

/home/jlin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Let's play around with variables in tensorflow!

In [4]:
# use tf.constant for constants
x = tf.constant(100, name = 'x')
y = tf.constant(200, name = 'x')

# use tf.Variable for things that are variables (can change)
dist = tf.Variable(abs(x - y), name = 'dist')

# create your global variables initializer
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(dist))

100


Now you know how to create variables, constants and the global variables initializer. You also know how to create a session, initialize your global variables, and run a variable

Writing a Tensorflow program consists of creating variables and defining operations between them, initializing your variables, then creating and running your session on your variables.

Now let's play with placeholders. You can create placeholders with `tf.placeholder`. When you run operations with a placeholder, you need a feed dictionary, which contains the values of those placeholders. Let's write the code above with placeholders now.

In [10]:
x = tf.placeholder(tf.float32, name = 'x')
y = tf.placeholder(tf.float32, name = 'y')

d = abs(x - y)

init = tf.global_variables_initializer()

xval = 100
yval = 200
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(d, feed_dict = {x: xval, y : yval}))

100.0
